In [ ]:
import geemap
import json
import os
import requests
import re
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd
import ee
import numpy as np
from glob import glob

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#initialize GEE using your Google Account
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
    

In [ ]:
poyang_point = [29.182232, 116.298122]
Map = geemap.Map(center=poyang_point ,zoom=9)

#poyang test site
file_path = './data/boundaries/poyang.geojson'
with open(file_path) as f:
    json_data = json.load(f)
    ee_data = geojson_to_ee(json_data)
    poly_line = ee.Geometry.LineString(ee_data.geometry().coordinates().get(0))
Map.addLayer(ee_object=poly_line ,vis_params={'color':'red'}, name="Poyang")
Map

In [ ]:
cl_pct = 60

poyang_geom = ee.Geometry.Point(poyang_point[::-1])

l8_col = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
    .filterBounds(poyang_geom).filterDate('2015-01-01', '2015-12-31') \
    .filter(ee.Filter.lte('CLOUD_COVER', cl_pct))

l8_col_list = l8_col.toList(l8_col.size())
print('Nr. images found: ', l8_col_list.size().getInfo())


In [ ]:
def plot_rgb(img,text):
    viz_rgb = {'bands': ['B4', 'B3', 'B2'],'gain': [0.1, 0.1, 0.1],'scale':90}
    Map.addLayer(img, viz_rgb, text+' RGB')
    
def plot_false(img,text):
    viz_rgb = {'bands': ['B6', 'B5', 'B2'],'gain': [0.1, 0.1, 0.1],'scale':90}
    Map.addLayer(img, viz_rgb, text+' False')
    
img = ee.Image(l8_col_list.get(7))
plot_rgb(img,img.getInfo()['id'])
plot_false(img,img.getInfo()['id'])


In [ ]:
def add_wiw(img):
    wiw = img.expression(
        '((B8A/10000) <= 0.1804) && ((B12/10000) <= 0.1131)',{'B8A':img.select('B8A'),'B12':img.select('B12')}
    ).rename('RWS')
    return img.addBands(wiw)

def add_rws(img):
    mndwi = img.normalizedDifference(['B3','B6']).rename('MNDWI')
    mgrn = img.select(['B3','B4','B5']).subtract(10000).reduce(ee.Reducer.min()).rename('MGRN')
    rws = img.expression(
        '(MNDWI > 0.3) && (MGRN < 0.15)',{'MNDWI':mndwi,'MGRN':mgrn}
    ).rename('RWS')
    return img.addBands(rws)

def apply_rws(img):
    mask = img.select('RWS').eq(1)
    masked_img = img.updateMask(mask)
    return masked_img


def get_labels(img):
    cluster =  img.select('cluster')
    cluster_vector = cluster.reduceToVectors(reducer = ee.Reducer.countEvery(),geometry=img.geometry(),bestEffort=True)
    labels = ee.List(cluster_vector.aggregate_array('label')).distinct().sort()
    return labels


def add_kmeans(img):
    
    rws_region = img.select('RWS') \
        .reduceToVectors(reducer = ee.Reducer.countEvery(),scale=900,bestEffort=True,geometryType='polygon') \
        .filter(ee.Filter.eq("label", 1)).union(1)
    
    rgb = img.select(['B4', 'B3', 'B2'])
    train_samples = rgb.sample(scale=30,region=rws_region,numPixels=3000,seed=42)
    k_means = ee.Clusterer.wekaKMeans(8).train(train_samples)
    img_cluster = rgb.cluster(k_means)

    rws_mask = img.select('RWS').eq(1)
    img_cluster_masked = img_cluster.updateMask(rws_mask).rename('cluster')
    
    return img.addBands(img_cluster_masked)

def plot_mask(data,layer_name):
    img = data.select(layer_name)
    img_mask = img.updateMask(img.eq(1))
    Map.addLayer(img_mask, {'palette': 'red'}, f'{layer_name}')

In [ ]:
def add_mnws(img):
    img_cluster = img.select('cluster')
    labels = list(range(0,8))

    nws_list = []
    
    for label in labels:
        cluster_region = img_cluster.eq(ee.Number(label))
        
        scores = []
        band_names = ['B7','B6','B5','B4','B3','B2']
        
        score_sum = None
        for band in band_names:
            
            img_band_raw = img.select(band)
            
            img_band = img.select(band).updateMask(cluster_region)
            reducers = ee.Reducer.mean().combine(reducer2 = ee.Reducer.stdDev(),sharedInputs= True)
            img_stats = img_band.reduceRegion(reducer= reducers,bestEffort= True)
            img_band_mu = ee.Number(img_stats.get(f'{band}_mean'))
            img_band_std = ee.Number(img_stats.get(f'{band}_stdDev'))
            img_band_score = img_band_raw.subtract(img_band_mu).divide(img_band_std).pow(2).rename(f'{label} {band}')
            scores.append(img_band_score)
            
        nws = ee.Image.cat(scores).reduce('sum').divide(ee.Number(len(band_names))).sqrt().rename('NWS')
        nws_list.append(nws)
        
    mnws = ee.Image.cat(nws_list).reduce('min').rename('MNWS')

    return img.addBands(mnws)

def get_wd(img):
    img_mnws = img.select('MNWS')
    wd_i = img.expression('MNWS < 3',{'MNWS':img_mnws})
    return wd_i

In [ ]:
# Map.addLayer(mnws_img, {'palette':['F94144','F3722C','F8961E','F9C74F','90BE6D','43AA8B','577590'],'min':0,'max':10})

In [ ]:
mnws_ts = l8_col.map(add_rws).map(add_kmeans).map(add_mnws)
img_wf = mnws_ts.map(get_wd).reduce('sum').divide(10)

In [ ]:
Map.addLayer(img_wf, {'palette':['white', '0000FF'],'min':0.1,'max':1})

In [ ]:
#export all bands stack to one geotiff
geemap.ee_export_image(img_wf, filename='wf.tif', scale=90, region=poly_line, file_per_band=False)
remove_zips = list(map(lambda x:os.remove(x),glob('*.zip')))